__Due Date:__ 2022-10-16 at 8:30 AM PT

---


__Name:__ \<your name here\>



For this assignment, you will practice downloadings, cleaning, and analyzing data from the [National Risk Index (NRI)](https://hazards.fema.gov/nri/) and the [CDC Social Vulnerability Index (SVI)](https://www.atsdr.cdc.gov/placeandhealth/svi/index.html).

## Preparation

1. Create a 'data' folder in the root directory of your repository.
1. Inside the 'data' folder, create a 'raw' folder.
1. Add and commit a '.gitignore' file to the root directory of this repository that excludes all contents of the 'data' folder.
1. Download the county-level NRI and SVI data for the entire United States. Place the data in the 'data/raw' folder.
    - NRI data: Download the "All Counties - County-level detail (Table)" CSV file from [here](https://hazards.fema.gov/nri/data-resources#csvDownload).
    - SVI data: Download the 2022 county-level dataset (CSV format) from [here](https://www.atsdr.cdc.gov/place-health/php/svi/svi-data-documentation-download.html).
1. In the repository README, provide a brief (1-2 sentence) description of each file in the 'data' folder and a link to the original source of the data.

## Task 1 - NRI Data Cleaning

__1. Import the NRI data. Ensure that the [FIPS code](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code) variable ('STCOFIPS') is correctly identified as a string / character variable. Otherwise, the leading zeros will be removed.__

In [ ]:
import pandas as pd

nri_path = "data/raw/NRI_Table_Counties.csv"
svi_path = "data/raw/SVI_2022_US_county.csv"

nri = pd.read_csv(nri_path, dtype={"STCOFIPS": str})
svi = pd.read_csv(svi_path, dtype={"FIPS": str})

__2. Subset the NRI data to include only the 5-digit state/county FIPS code and all colums ending with '\_AFREQ' and '\_RISKR'. Each of these columns represents a different hazard type.__

In [ ]:
afreq_cols = [col for col in nri.columns if col.endswith('_AFREQ')]
riskr_cols = [col for col in nri.columns if col.endswith('_RISKR')]

nri_subset = nri[['STCOFIPS'] + afreq_cols + riskr_cols]
nri_subset.head()

__3. Create a table / dataframe that, for each hazard type, shows the number of missing values in the '\_AFREQ' and '\_RISKR' columns.__

In [ ]:
missing_values = nri_subset.isnull().sum()
missing_values

__4. Create a new column in the original data table indicating whether or not 'AVLN_AFREQ' is missing or observed. Show the cross-tabulation of the 'AVLN_AFREQ' missingness and 'AVLN_RISKR' columns (including missing values). What do you observe?__

In [ ]:
nri_subset['AVLN_AFREQ_missing'] = nri_subset['AVLN_AFREQ'].isnull()
crosstab = pd.crosstab(nri_subset['AVLN_AFREQ_missing'], nri_subset['AVLN_RISKR'], dropna=False)
crosstab

__5. Assuming that a risk that is "not applicable" to a county has an annualized frequency of 0, impute the relevant missing values in the '\_AFREQ' columns with 0.__

In [ ]:
hazards = [col.split('_')[0] for col in afreq_cols]

for hazard in hazards:
    afreq_col = f"{hazard}_AFREQ"
    riskr_col = f"{hazard}_RISKR"

    na_rows = nri_subset[riskr_col] == 'Not Applicable'

    nri_subset.loc[na_rows, afreq_col] = nri_subset.loc[na_rows, afreq_col].fillna(0)

## Task 2 - SVI Data Cleaning

__1. Import the SVI data. Ensure that the FIPS code is correctly identified as a string / character variable. Otherwise, the leading zeros will be removed.__
__1. Subset the SVI data to include only the following columns:__
`ST, STATE, ST_ABBR, STCNTY, COUNTY, FIPS, LOCATION, AREA_SQMI, E_TOTPOP, EP_POV150, EP_UNEMP, EP_HBURD, EP_NOHSDP, EP_UNINSUR, EP_AGE65, EP_AGE17, EP_DISABL, EP_SNGPNT, EP_LIMENG, EP_MINRTY, EP_MUNIT, EP_MOBILE, EP_CROWD, EP_NOVEH, EP_GROUPQ, EP_NOINT, EP_AFAM, EP_HISP, EP_ASIAN, EP_AIAN, EP_NHPI, EP_TWOMORE, EP_OTHERRACE`


In [ ]:
svi_columns = [
    'ST', 'STATE', 'ST_ABBR', 'STCNTY', 'COUNTY', 'FIPS', 'LOCATION', 'AREA_SQMI',
    'E_TOTPOP', 'EP_POV150', 'EP_UNEMP', 'EP_HBURD', 'EP_NOHSDP', 'EP_UNINSUR',
    'EP_AGE65', 'EP_AGE17', 'EP_DISABL', 'EP_SNGPNT', 'EP_LIMENG', 'EP_MINRTY',
    'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ', 'EP_NOINT',
    'EP_AFAM', 'EP_HISP', 'EP_ASIAN', 'EP_AIAN', 'EP_NHPI', 'EP_TWOMORE', 'EP_OTHERRACE'
]

svi_subset = svi[svi_columns]

__2. Create a table / dataframe that shows the number of missing values in each column.
(Hint: if you wrote a function for Task 1, you can reuse it here.)__

In [ ]:
svi_missing_values = svi_subset.isnull().sum()
svi_missing_values

## Task 3 - Data Merging
__1. Identify any FIPS codes that are present in the NRI data but not in the SVI data and vice versa. Describe any discrepancies and possible causes? What to these discrepancies, if any, mean for interpreting results based on the merged dataset moving forward?__

In [ ]:
nri_fips = set(nri_subset['STCOFIPS'])
svi_fips = set(svi_subset['FIPS'])

nri_not_in_svi = nri_fips - svi_fips
svi_not_in_nri = svi_fips - nri_fips

print(f"FIPS in NRI but not in SVI: {nri_not_in_svi}")
print("\n")
print(f"FIPS in SVI but not in NRI: {svi_not_in_nri}")

__2. Merge the NRI and SVI data on the FIPS code. Use an outer join to keep all counties in the final dataset.__

In [ ]:
merged_data = pd.merge(nri_subset, svi_subset, left_on='STCOFIPS', right_on='FIPS', how='outer')
merged_data

__3. Create a table / dataframe that shows the number of missing values in each column of the merged dataset.__

In [ ]:
merged_missing_values = merged_data.isnull().sum()
merged_missing_values

## Task 4 - Data Analysis

__1. For each numerical variable in the merged dataset, plot a histogram showing the distribution of values.
(Hint: write a function to make the histogram for a single variable, then use a loop or apply function to make the histograms for all numerical variables.)__

In [ ]:
import matplotlib.pyplot as plt

def plot_histogram(column):
    plt.figure(figsize=(8, 6))
    plt.hist(merged_data[column].dropna(), bins=30, edgecolor='k', alpha=0.7)
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.show()

numerical_cols = merged_data.select_dtypes(include=['number']).columns
for col in numerical_cols:
    plot_histogram(col)

In [ ]:
merged_data.to_csv("data/processed/nri_svi_merged.csv")